# Multilingual Text-to-Speech

## Install Dependencies

In [ ]:
import sys
import os
import IPython
from IPython.display import Audio

In [ ]:
PROJECT_ROOT = os.getcwd()

In [ ]:
! pip install -q --user soundfile
! pip install -q --user phonemizer
! pip install -q --user epitran
! pip install --user protobuf==3.20.3

In [ ]:
!pip install -r requirements.txt --user
!pip install --user --upgrade librosa

## Setup Dataset

### Downloads COMVOI - Dutch, German, French, Russian and Chinese audio.

In [ ]:
%%capture

!rm -rf data/comvoi
!curl -O -L https://github.com/Tomiinek/Multilingual_Text_to_Speech/releases/download/v1.0/comvoi.zip
!unzip -q comvoi.zip -d data/comvoi
!rm comvoi.zip

### Downloads VCTK - English audio.

In [ ]:
%%capture

!rm -rf data/vctk
!curl -O -L http://studio336.sk/css/vctk.zip
!mkdir data/vctk
!unzip vctk.zip -d data/vctk
!rm vctk.zip

### Remove audios in meta.csv files that do not exist.

In [ ]:
import shutil

base_dir = 'data/comvoi'
for lang_dir in os.listdir(base_dir):
    # Rename "meta.csv" files to "_meta.csv".
    os.rename(os.path.join(base_dir, lang_dir, 'meta.csv'), os.path.join(base_dir, lang_dir, '_meta.csv'))
    
    with open(os.path.join(base_dir, lang_dir, '_meta.csv'), mode = 'r', encoding = 'utf-8') as meta_csv:
        lines = meta_csv.readlines()
        copy_of_lines = lines.copy()
        
        # Example line: 04|common_voice_fr_18576291.wav|Que suis-je auprès de Lui.
        for line in lines:
            audio_file_subdir = line.split('|')[0]
            audio_file_name = line.split('|')[1]
            
            # Example audio_file_path: data/comvoi/fr/wavs/04/common_voice_fr_18576287.wav
            audio_file_path = os.path.join(base_dir, lang_dir, 'wavs', audio_file_subdir, audio_file_name)
            if not os.path.exists(audio_file_path):
                copy_of_lines.remove(line)

    with open(os.path.join(base_dir, lang_dir, 'meta.csv'), mode = 'w', encoding = 'utf-8') as meta_csv:
        meta_csv.writelines(copy_of_lines)

### Creates a train.txt file that stores information for each dataset.
### Also generates spectrograms for each audio file.

In [ ]:
import dataset.dataset as ds
ds.TextToSpeechDataset.create_meta_file('my_common_voice', 'data/comvoi', 'train.txt', 22050, 1102, True, False)

In [ ]:
import dataset.dataset as ds
ds.TextToSpeechDataset.create_meta_file('vctk', 'data/vctk/VCTK-Corpus/VCTK-Corpus', 'train.txt', 22050, 1102, True, False)

In [ ]:
# Combine COMVOI and VCTK train.txt files

!cat data/comvoi/train.txt data/vctk/VCTK-Corpus/VCTK-Corpus/train.txt > data/comvoi_vctk/train.txt

### Ensure that file paths are relative to data/comvoi_vctk directory.

In [ ]:
# Back up the train.txt file first.
!cp data/comvoi_vctk/train.txt data/comvoi_vctk/_train.txt

In [ ]:
COMVOI_LANGUAGES = ['de', 'fr', 'nl', 'ru', 'zh']
VCTK_LANGUAGES = ['en-us']

with open('data/comvoi_vctk/_train.txt', mode = 'r', encoding = 'utf-8') as file:
    lines = file.readlines()
    for i in range(len(lines)):
        line = lines[i]
        line_parts = line.split('|')
        lang = line_parts[2]
        if lang in COMVOI_LANGUAGES:
            # Change e.g. "de/wavs/01/common_voice_de_18362579.wav" to "../comvoi/de/wavs/01/common_voice_de_18362579.wav"
            line_parts[3] = os.path.join('../comvoi/', line_parts[3])
            line_parts[4] = os.path.join('../comvoi/', line_parts[4])
            line_parts[5] = os.path.join('../comvoi/', line_parts[5])
            lines[i] = '|'.join(line_parts)
        
        if lang in VCTK_LANGUAGES:
            line_parts[3] = os.path.join('../vctk/VCTK-Corpus/VCTK-Corpus/', line_parts[3])
            line_parts[4] = os.path.join('../vctk/VCTK-Corpus/VCTK-Corpus/', line_parts[4])
            line_parts[5] = os.path.join('../vctk/VCTK-Corpus/VCTK-Corpus/', line_parts[5])
            lines[i] = '|'.join(line_parts)

    with open('data/comvoi_vctk/train.txt', mode = 'w', encoding = 'utf-8') as file:
        file.writelines(lines)

In [ ]:
!rm data/comvoi_vctk/_train.txt

### Split data into train.txt and val.txt.

In [ ]:
# Back up the train.txt file first.
!cp data/comvoi_vctk/train.txt data/comvoi_vctk/_train.txt

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split

with open('data/comvoi_vctk/_train.txt', mode = 'r', encoding = 'utf-8') as file:
    all_lines = file.readlines()
    train, val = train_test_split(all_lines, test_size=0.15, random_state=42)
    print(len(train))
    print(len(val))
        
    with open('data/comvoi_vctk/train.txt', mode = 'w', encoding = 'utf-8') as file:
        file.writelines(train)
    with open('data/comvoi_vctk/val.txt', mode = 'w', encoding = 'utf-8') as file:
        file.writelines(val)

In [ ]:
!rm data/comvoi_vctk/_train.txt

## Train

In [ ]:
os.chdir(PROJECT_ROOT)

!PYTHONIOENCODING=utf-8 python3 train.py --hyper_parameters generated_switching_comvoi_vctk

### Zip logs

In [ ]:
%cd ~/Multilingual_Text_to_Speech
!rm logs.zip
!zip logs.zip -r logs

## Test

In [ ]:
!echo "fr|Cette requête s'explique par les relations peu conventionnelles que Schrödinger entretient avec les femmes.|01-zh|fr"  | python3 synthesize.py --checkpoint checkpoints/GENERATED-SWITCHING_loss-3-0.375 --save_spec
!echo "ru|Как считают современные археологи, на месте находились четыре различных храма.|01-zh|ru"  | python3 synthesize.py --checkpoint checkpoints/GENERATED-SWITCHING_loss-3-0.375 --save_spec
!echo "de|Sie liegt zwischen dem Ijsselmeer, der Ijssel und den Hügeln der Veluwe.|01-zh|de"  | python3 synthesize.py --checkpoint checkpoints/GENERATED-SWITCHING_loss-3-0.375 --save_spec
!echo "zh|wǒ zài nián qīng shí hou yě céng jīng zuò guò xǔ duō mèng.|01-zh|zh"  | python3 synthesize.py --checkpoint checkpoints/GENERATED-SWITCHING_loss-3-0.375 --save_spec
!echo "en|Now, the way I would do it is by better analysing the data.|01-zh|en-us"  | python3 synthesize.py --checkpoint checkpoints/GENERATED-SWITCHING_loss-3-0.375 --save_spec

import IPython.display as ipd
ipd.display(ipd.Audio('fr.wav'))
ipd.display(ipd.Audio('ru.wav'))
ipd.display(ipd.Audio('de.wav'))
ipd.display(ipd.Audio('zh.wav'))
ipd.display(ipd.Audio('en.wav'))